In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator 
plt.style.use('fivethirtyeight')    # 样式美化定制画布风格
%matplotlib inline    
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')    # 设定显示图片的分辨率
import warnings
warnings.filterwarnings("ignore")  

In [ ]:
delimiter="\t"
confirmed_df = pd.read_csv('../input/covid19uspredict/case_total_and_rate_per_100000__united_states.csv',delimiter="\t")
deaths_df = pd.read_csv('../input/covid19uspredict/death_total_and_rate_per_100000__united_states.csv',delimiter="\t")

In [ ]:
confirmed_df.head()

In [ ]:
deaths_df.head()

In [ ]:
confirmed = confirmed_df[2:444]
# print(confirmed)
deaths = deaths_df[2:444]
# print(deaths)

In [ ]:
confirmed['Date'] = confirmed['Case Total and Rate per 100000 - United States'].map(lambda x:x.split(',')[0])
confirmed['Case Total'] = confirmed['Case Total and Rate per 100000 - United States'].map(lambda x:x.split(',')[1])
confirmed['Rate per 100000'] = confirmed['Case Total and Rate per 100000 - United States'].map(lambda x:x.split(',')[2])
print(confirmed)

In [ ]:
deaths['Date'] = deaths['Death Total and Rate per 100000 - United States'].map(lambda x:x.split(',')[0])
deaths['Death Total'] = deaths['Death Total and Rate per 100000 - United States'].map(lambda x:x.split(',')[1])
deaths['Rate per 100000'] = deaths['Death Total and Rate per 100000 - United States'].map(lambda x:x.split(',')[2])
print(deaths)

In [ ]:
confirmed['Date']

In [ ]:
dates = []
us_cases = []
us_deaths = []
dates.append(confirmed['Date'])
# dates = np.array(dates)
us_cases.append(confirmed['Case Total'].astype('int'))
us_deaths.append(deaths['Death Total'].astype('int'))
# print(dates)

In [ ]:
type(confirmed['Case Total'])

In [ ]:
days_since_1_22 = np.array([i for i in range(len(dates[0]))]).reshape(-1, 1)
us_cases = np.array(us_cases).reshape(-1, 1)
us_deaths = np.array(us_deaths).reshape(-1, 1)
us_cases = us_cases[::-1]
us_deaths = us_deaths[::-1]
# days_since_1_22

In [ ]:
days_in_future = 10
future_forcast = np.array([i for i in range(len(dates[0])+days_in_future)]).reshape(-1, 1)
adjusted_dates = future_forcast[:-10]
# future_forcast

In [ ]:
start = '1/22/2020'
start_date = datetime.datetime.strptime(start, '%m/%d/%Y')
# print(start_date)
future_forcast_dates = []
for i in range(len(future_forcast)):
    future_forcast_dates.append((start_date + datetime.timedelta(days=i)).strftime('%m/%d/%Y'))
# print(future_forcast_dates) 

In [ ]:
X_train_confirmed, X_test_confirmed, y_train_confirmed, y_test_confirmed = train_test_split(days_since_1_22[50:], us_cases[50:], test_size=0.02,shuffle=False)

In [ ]:
X_train_death, X_test_death, y_train_death, y_test_death = train_test_split(days_since_1_22[50:], us_deaths[50:], test_size=0.02,shuffle=False)

In [ ]:
# print(X_train_confirmed)
# print(y_train_confirmed)
# print(X_test_confirmed)
print(y_test_confirmed)

In [ ]:
svm_confirmed = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_confirmed.fit(X_train_confirmed, y_train_confirmed)    # 用训练数据拟合模型
svm_pred = svm_confirmed.predict(future_forcast)
# print(svm_pred)

In [ ]:
svm_death = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=3, C=0.1)
svm_death.fit(X_train_death, y_train_death)    # 用训练数据拟合模型
svm_pred1 = svm_death.predict(future_forcast)
# print(svm_pred_death)

In [ ]:
svm_test_pred = svm_confirmed.predict(X_test_confirmed)
plt.plot(y_test_confirmed)
plt.plot(svm_test_pred)
plt.legend(['Test Data', 'SVM Predictions'])
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.title("Cases测试集对比", color = 'white')
print('MAE:%e' %mean_absolute_error(y_test_confirmed, svm_test_pred))
# print('MAE:', np.mean(np.abs(y_test_confirmed-svm_test_pred)))
print('MSE:%e' %mean_squared_error(y_test_confirmed, svm_test_pred))
# print('MSE:',np.mean((y_test_confirmed-svm_test_pred)**2))

In [ ]:
svm_test_pred1 = svm_death.predict(X_test_death)
plt.plot(y_test_death)
plt.plot(svm_test_pred1)
plt.legend(['Test Data', 'SVM Predictions'])
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.title("Deaths测试集对比", color = 'white')
print('MAE:%e' %mean_absolute_error(y_test_death, svm_test_pred1))
# print('MAE:', np.mean(np.abs(y_test_confirmed-svm_test_pred)))
print('MSE:%e' %mean_squared_error(y_test_death, svm_test_pred1))
# print('MSE:',np.mean((y_test_confirmed-svm_test_pred)**2))

In [ ]:
# 把数据转换成多项式回归
poly = PolynomialFeatures(degree=4)    # degree是多项式特征的次数(阶数)
poly_X_train_confirmed = poly.fit_transform(X_train_confirmed)
poly_X_test_confirmed = poly.fit_transform(X_test_confirmed)
poly_future_forcast = poly.fit_transform(future_forcast)

bayesian_poly = PolynomialFeatures(degree=5)
bayesian_poly_X_train_confirmed = bayesian_poly.fit_transform(X_train_confirmed)
bayesian_poly_X_test_confirmed = bayesian_poly.fit_transform(X_test_confirmed)
bayesian_poly_future_forcast = bayesian_poly.fit_transform(future_forcast)

In [ ]:
# polynomial regression
linear_model = LinearRegression(normalize=True, fit_intercept=False)
linear_model.fit(poly_X_train_confirmed, y_train_confirmed)
test_linear_pred = linear_model.predict(poly_X_test_confirmed)
linear_pred = linear_model.predict(poly_future_forcast)
print('MAE:%e' %mean_absolute_error(y_test_confirmed, test_linear_pred))
print('MSE:%e' %mean_squared_error(y_test_confirmed, test_linear_pred))

In [ ]:
print(linear_model.coef_)    # coef_代表的是模型参数，coef_包含用于预测每个目标的系数

In [ ]:
plt.plot(y_test_confirmed)
plt.plot(test_linear_pred)
plt.legend(['Test Data', 'Polynomial Regression Predictions'])
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.title("Cases测试集对比",color = 'white')

In [ ]:
# bayesian ridge polynomial regression
tol = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
alpha_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
alpha_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_1 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
lambda_2 = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
normalize = [True, False]

bayesian_grid = {'tol': tol, 'alpha_1': alpha_1, 'alpha_2' : alpha_2, 'lambda_1': lambda_1, 'lambda_2' : lambda_2, 
                 'normalize' : normalize}

bayesian = BayesianRidge(fit_intercept=False)
bayesian_search = RandomizedSearchCV(bayesian, bayesian_grid, scoring='neg_mean_squared_error', cv=3, return_train_score=True, n_jobs=-1, n_iter=40, verbose=1)
bayesian_search.fit(bayesian_poly_X_train_confirmed, y_train_confirmed)

In [ ]:
bayesian_search.best_params_

In [ ]:
bayesian_confirmed = bayesian_search.best_estimator_
test_bayesian_pred = bayesian_confirmed.predict(bayesian_poly_X_test_confirmed)
bayesian_pred = bayesian_confirmed.predict(bayesian_poly_future_forcast)
print('MAE:%e' %mean_absolute_error(y_test_confirmed, test_bayesian_pred))
print('MSE:%e' %mean_squared_error(y_test_confirmed, test_bayesian_pred))

In [ ]:
plt.plot(y_test_confirmed)
plt.plot(test_bayesian_pred)
plt.legend(['Test Data', 'Bayesian Ridge Polynomial Predictions'])
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.title("Cases测试集对比", color = 'white')

In [ ]:
def plot_predictions(x, y, pred, algo_name, color):
    plt.figure(figsize=(16, 10))
    plt.plot(x, y)
    plt.plot(future_forcast, pred, linestyle='dashed', color=color)
    plt.title('US Coronavirus Cases Over Time', size=30)
    plt.xlabel('Days Since 1/22/2020', size=30)
    plt.ylabel('US of Cases', size=30)
    plt.legend(['Confirmed Cases', algo_name], prop={'size': 20})
    plt.xticks(size=20)
    plt.yticks(size=20)
    plt.show()

In [ ]:
plot_predictions(adjusted_dates, us_cases, np.array(svm_pred).reshape(-1, 1), 'SVM Predictions', 'purple')

In [ ]:
plot_predictions(adjusted_dates, us_deaths, np.array(svm_pred1).reshape(-1, 1), 'SVM Predictions', 'purple')

In [ ]:
plot_predictions(adjusted_dates, us_cases, linear_pred, 'Polynomial Regression Predictions', 'orange')

In [ ]:
plot_predictions(adjusted_dates, us_cases, bayesian_pred, 'Bayesian Ridge Regression Predictions', 'green')

In [ ]:
# Future predictions using SVM 
svm_df = pd.DataFrame({'Date': future_forcast_dates[-10:], 'SVM Predicted US of Confirmed Cases': np.round(svm_pred[-10:])})
svm_df.style.background_gradient(cmap='Reds')

In [ ]:
svm_df1 = pd.DataFrame({'Date': future_forcast_dates[-10:], 'SVM Predicted US of Deaths': np.round(svm_pred1[-10:])})
svm_df1.style.background_gradient(cmap='Blues')

In [ ]:
# Future predictions using polynomial regression
linear_pred = linear_pred.reshape(1,-1)[0]
linear_df = pd.DataFrame({'Date': future_forcast_dates[-10:], 'Polynomial Predicted US of Confirmed Cases': np.round(linear_pred[-10:])})
linear_df.style.background_gradient(cmap='Reds')

In [ ]:
# Future predictions using Bayesian Ridge 
bayesian_df = pd.DataFrame({'Date': future_forcast_dates[-10:], 'Bayesian Ridge Predicted US of Confirmed Cases': np.round(bayesian_pred[-10:])})
bayesian_df.style.background_gradient(cmap='Reds')